In [18]:
import spacy
from spacy.tokens import DocBin
nlp = spacy.load("en_core_web_sm")
db = DocBin()

In [2]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-28 07:19:43.069730: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-28 07:19:43.069856: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-28 07:19:43.069879: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [3]:
import json

training_data = json.load(open("train_annot.json"))

for text,annot in training_data['annotations']:
    doc = nlp.make_doc(text)
    ents = []
    for start,end,label in annot['entities']:
        span = doc.char_span(start,end,label=label,alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
        
    doc.ents = ents
    db.add(doc)
    
db.to_disk("./train.spacy")

In [19]:
import json

training_data = json.load(open("test_annotations.json"))

for text,annot in training_data['annotations']:
    doc = nlp.make_doc(text)
    ents = []
    for start,end,label in annot['entities']:
        span = doc.char_span(start,end,label=label,alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
        
    doc.ents = ents
    db.add(doc)
    
db.to_disk("./test.spacy")

In [20]:
!python -m spacy train ./config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy 


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-28 07:39:35.446529: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-28 07:39:35.446703: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-28 07:39:35.446736: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [21]:
best = spacy.load(r"./output/model-best") #load the best model

In [22]:

doc = best("Apple has a great new product called Homepod")
spacy.displacy.render(doc, style="ent", jupyter=True)


In [23]:
doc = best("Steam has a new console SteamDeck which can run Counter Strike too")
spacy.displacy.render(doc, style="ent", jupyter=True)

In [41]:
doc = best("Cricket vs Hockey which is the more popular sport in India")
spacy.displacy.render(doc, style="ent", jupyter=True)

In [39]:
doc = best("Tim Cook has invited Serena Williams for the Apple product launch")
spacy.displacy.render(doc, style="ent", jupyter=True)